# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
import datetime
from sql_queries import *



In [ ]:
#%run etl.py

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [2]:
# to run create tables
%run create_tables.py

In [3]:
%run sql_queries.py

In [7]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
#to run etl.py
%run etl.py

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [8]:
song_files = get_files('data/song_data/')
len(song_files)

71

In [9]:
# select first song on list
filepath = song_files[0]

In [10]:
df = pd.read_json(filepath, lines=True)
df.head()

artist_id  artist_latitude artist_location  artist_longitude  \
0  AR8IEZO1187B99055E              NaN                               NaN   

    artist_name   duration  num_songs             song_id          title  year  
0  Marc Shaiman  149.86404          1  SOINLJW12A8C13314C  City Slickers  2008

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
# convert array to list
df.values[0].tolist()

['AR8IEZO1187B99055E',
 nan,
 '',
 nan,
 'Marc Shaiman',
 149.86404,
 1,
 'SOINLJW12A8C13314C',
 'City Slickers',
 2008]

In [12]:
song_data =  df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

In [13]:
song_data = (song_data[0], song_data[1], song_data[2], song_data[3], song_data[4])
song_data

('SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
song_table_insert

'INSERT INTO songs(song_id, title, artist_id, year, duration)\n                        VALUES(%s, %s, %s, %s, %s)\n                        ON CONFLICT (song_id) \n                        DO NOTHING'

In [15]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [16]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data = (artist_data[0], artist_data[1], artist_data[2], artist_data[3], artist_data[4])
artist_data


('AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [20]:
log_files = get_files("data/log_data/")

In [21]:
filepath = log_files[0]
filepath 

'/home/workspace/data/log_data/2018/11/2018-11-29-events.json'

In [22]:
df = pd.read_json(filepath, lines=True)
df.head()

artist       auth firstName gender  itemInSession lastName  \
0  Sydney Youngblood  Logged In     Jacob      M             53    Klein   
1         Gang Starr  Logged In     Layla      F             88  Griffin   
2              3OH!3  Logged In     Layla      F             89  Griffin   
3        RÃÂ¶yksopp  Logged In     Jacob      M             54    Klein   
4         Kajagoogoo  Logged In     Layla      F             90  Griffin   

      length level                             location method      page  \
0  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
1  151.92771  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
2  192.52200  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
3  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  223.55546  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   

   registration  sessionId                                         song  \
0  1.540558e+12        954                            Ain't No Sunshine   
1  1.541057e+12        984                   My Advice 2 You (Explicit)   
2  1.541057e+12        984  My First Kiss (Feat. Ke$ha) [Album Version]   
3  1.540558e+12        954                       The Girl and The Robot   
4  1.541057e+12        984                                      Too Shy   

   status             ts                                          userAgent  \
0     200  1543449657796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
1     200  1543449690796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
2     200  1543449841796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
3     200  1543449895796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
4     200  1543450033796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   

  userId  
0     73  
1     24  
2     24  
3     73  
4     24

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [23]:
df = df[df['page']=="NextSong"].reset_index()
df.head()

index             artist       auth firstName gender  itemInSession  \
0      0  Sydney Youngblood  Logged In     Jacob      M             53   
1      1         Gang Starr  Logged In     Layla      F             88   
2      2              3OH!3  Logged In     Layla      F             89   
3      3        RÃÂ¶yksopp  Logged In     Jacob      M             54   
4      4         Kajagoogoo  Logged In     Layla      F             90   

  lastName     length level                             location method  \
0    Klein  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL    PUT   
1  Griffin  151.92771  paid         Lake Havasu City-Kingman, AZ    PUT   
2  Griffin  192.52200  paid         Lake Havasu City-Kingman, AZ    PUT   
3    Klein  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL    PUT   
4  Griffin  223.55546  paid         Lake Havasu City-Kingman, AZ    PUT   

       page  registration  sessionId  \
0  NextSong  1.540558e+12        954   
1  NextSong  1.541057e+12        984   
2  NextSong  1.541057e+12        984   
3  NextSong  1.540558e+12        954   
4  NextSong  1.541057e+12        984   

                                          song  status             ts  \
0                            Ain't No Sunshine     200  1543449657796   
1                   My Advice 2 You (Explicit)     200  1543449690796   
2  My First Kiss (Feat. Ke$ha) [Album Version]     200  1543449841796   
3                       The Girl and The Robot     200  1543449895796   
4                                      Too Shy     200  1543450033796   

                                           userAgent userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     24  
2  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     24  
3  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73  
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     24

In [24]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [25]:
df['week'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).isocalendar()[1])
df['week_day'] = t.apply(lambda x: datetime.date(x.year, x.month, x.day).strftime("%A"))
df.head()

index             artist       auth firstName gender  itemInSession  \
0      0  Sydney Youngblood  Logged In     Jacob      M             53   
1      1         Gang Starr  Logged In     Layla      F             88   
2      2              3OH!3  Logged In     Layla      F             89   
3      3        RÃÂ¶yksopp  Logged In     Jacob      M             54   
4      4         Kajagoogoo  Logged In     Layla      F             90   

  lastName     length level                             location    ...     \
0    Klein  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL    ...      
1  Griffin  151.92771  paid         Lake Havasu City-Kingman, AZ    ...      
2  Griffin  192.52200  paid         Lake Havasu City-Kingman, AZ    ...      
3    Klein  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL    ...      
4  Griffin  223.55546  paid         Lake Havasu City-Kingman, AZ    ...      

       page  registration  sessionId  \
0  NextSong  1.540558e+12        954   
1  NextSong  1.541057e+12        984   
2  NextSong  1.541057e+12        984   
3  NextSong  1.540558e+12        954   
4  NextSong  1.541057e+12        984   

                                          song status             ts  \
0                            Ain't No Sunshine    200  1543449657796   
1                   My Advice 2 You (Explicit)    200  1543449690796   
2  My First Kiss (Feat. Ke$ha) [Album Version]    200  1543449841796   
3                       The Girl and The Robot    200  1543449895796   
4                                      Too Shy    200  1543450033796   

                                           userAgent userId week  week_day  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73   48  Thursday  
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     24   48  Thursday  
2  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     24   48  Thursday  
3  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     73   48  Thursday  
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     24   48  Thursday  

[5 rows x 21 columns]

In [26]:
time_data = (t, t.dt.hour, t.dt.day, df.week, t.dt.month, t.dt.year, df.week_day)
column_labels = ['start_time','hour','day','week','month', 'year','weekday']

In [27]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()
df['start_time'] = t
df.head()

index             artist       auth firstName gender  itemInSession  \
0      0  Sydney Youngblood  Logged In     Jacob      M             53   
1      1         Gang Starr  Logged In     Layla      F             88   
2      2              3OH!3  Logged In     Layla      F             89   
3      3        RÃÂ¶yksopp  Logged In     Jacob      M             54   
4      4         Kajagoogoo  Logged In     Layla      F             90   

  lastName     length level                             location  \
0    Klein  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL   
1  Griffin  151.92771  paid         Lake Havasu City-Kingman, AZ   
2  Griffin  192.52200  paid         Lake Havasu City-Kingman, AZ   
3    Klein  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL   
4  Griffin  223.55546  paid         Lake Havasu City-Kingman, AZ   

            ...            registration sessionId  \
0           ...            1.540558e+12       954   
1           ...            1.541057e+12       984   
2           ...            1.541057e+12       984   
3           ...            1.540558e+12       954   
4           ...            1.541057e+12       984   

                                          song  status             ts  \
0                            Ain't No Sunshine     200  1543449657796   
1                   My Advice 2 You (Explicit)     200  1543449690796   
2  My First Kiss (Feat. Ke$ha) [Album Version]     200  1543449841796   
3                       The Girl and The Robot     200  1543449895796   
4                                      Too Shy     200  1543450033796   

                                           userAgent  userId week  week_day  \
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...      73   48  Thursday   
1  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...      24   48  Thursday   
2  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...      24   48  Thursday   
3  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...      73   48  Thursday   
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...      24   48  Thursday   

               start_time  
0 2018-11-29 00:00:57.796  
1 2018-11-29 00:01:30.796  
2 2018-11-29 00:04:01.796  
3 2018-11-29 00:04:55.796  
4 2018-11-29 00:07:13.796  

[5 rows x 22 columns]

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [29]:
user_df = df[['userId','firstName', 'lastName', 'gender', 'level']]
user_df.head()

userId firstName lastName gender level
0     73     Jacob    Klein      M  paid
1     24     Layla  Griffin      F  paid
2     24     Layla  Griffin      F  paid
3     73     Jacob    Klein      M  paid
4     24     Layla  Griffin      F  paid

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [30]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [31]:
song_select

'SELECT songs.song_id, artists.artist_id FROM songs \n                  JOIN artists ON  songs.artist_id=artists.artist_id\n                  WHERE songs.title=%s AND artists.name=%s AND songs.duration=%s;'

In [32]:
songplay_table_insert

'INSERT INTO songplays( start_time, user_id,level, artist_id, \n                            song_id, session_id, location, user_agent)\n                            VALUES(%s, %s, %s, %s, %s, %s, %s, %s)'

In [34]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (str(row.start_time), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [35]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.